In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [3]:
df.shape

(891, 12)

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df.drop('Cabin', axis=1, inplace=True)

In [7]:
df.shape

(891, 11)

In [8]:
df = df.loc[df.Embarked.notna(),['Survived','Pclass','Sex','Embarked']]

In [9]:
df.shape

(889, 4)

In [10]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [17]:
X = df.loc[:,['Pclass']]

In [12]:
y = df.Survived

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
logreg = LogisticRegression(solver='lbfgs')

In [22]:
from sklearn.model_selection import cross_val_score

In [25]:
cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()

0.6783406335301212

In [27]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [28]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [29]:
from sklearn.preprocessing import OneHotEncoder

In [31]:
ohe = OneHotEncoder(sparse=False)

In [32]:
ohe.fit_transform(df)

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.]])

In [35]:
ohe.categories_

[array([0, 1], dtype=int64),
 array([1, 2, 3], dtype=int64),
 array(['female', 'male'], dtype=object),
 array(['C', 'Q', 'S'], dtype=object)]

In [36]:
X = df.drop('Survived', axis=1)

In [37]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [38]:
from sklearn.compose import make_column_transformer

In [40]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex','Embarked']),
    remainder='passthrough')

In [41]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [42]:
from sklearn.pipeline import make_pipeline

In [43]:
pipe = make_pipeline(column_trans, logreg)

In [45]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [49]:
X_new = X.sample(5, random_state=7)
X_new

,Pclass,Sex,Embarked
169,3,male,S
217,2,male,S
60,3,male,C
53,2,female,S
453,1,male,C


In [50]:
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [51]:
pipe.predict(X_new)

array([0, 0, 0, 1, 1], dtype=int64)